In [74]:
just_checking_integrity=False

In [75]:
import gc
import pandas as pd
import scipy.sparse as sps

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
valid=sps.load_npz("drive/MyDrive/recsys2022-main/dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [78]:
dataset=None
dataset_max=None

for file in ["UCF_W","ICBF_W","Graph_W","ICF_W"]:

    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"drive/MyDrive/recsys2022-main/dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


#del dataset_max
del df_feats
del df_max

460
loading UCF_W
18588456
loaded UCF_W
0
loading ICBF_W
1268640
loaded ICBF_W
0
loading Graph_W
9381571
loaded Graph_W
0
loading ICF_W
20341641
loaded ICF_W


In [82]:
import ast
submission_df_LSTM=pd.read_csv("drive/MyDrive/recsys2022-main/dataset/candidates/NN/LSTM/train/candidates.csv")
if just_checking_integrity:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
else:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [83]:
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="inner")

dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")


In [84]:
if just_checking_integrity:
    dataset=dataset[:int(0.5*len(dataset))]

In [86]:
submission_df_LSTM.Session_Id.nunique()

81618

In [88]:
import numpy as np
sessions=np.unique(dataset["Session_Id"])
train=pd.DataFrame()
train["Session_Id"]=sessions
if just_checking_integrity:
    train["Item_ID"]=bought[:len(sessions)]
else:
    train["Item_ID"]=bought
train["target"]=1

In [89]:
gc.collect()

3773

In [90]:
dataset=dataset.merge(train,on=["Session_Id","Item_ID"],how="left")
dataset.fillna(0,inplace=True)
gc.collect()

0

In [91]:
dataset["target"]=dataset["target"].astype("int8")
gc.collect()

0

In [92]:
sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

dataset=dataset[dataset['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

In [93]:
NHITS=len(dataset['Session_Id'].unique())
HITRATE=NHITS/81600 #approximate value
HITRATE

0.4649264705882353

In [94]:
dataset.head(20)

,Session_Id,Item_ID,Score_UCF_W,Score_ICBF_W,Score_Graph_W,Score_ICF_W,score_LSTM,rank_LSTM,Max_Score_UCF_W,Max_Score_ICBF_W,Max_Score_Graph_W,Max_Score_ICF_W,target
43,918383,55,0.054177,0.0,0.005355,0.018442,0.002479,31,594.02826,0.0,91.79825,1.092114,0
44,918383,189,0.129413,0.0,0.020602,0.278846,0.001137,54,594.02826,0.0,91.79825,1.092114,0
45,918383,379,0.046153,0.0,0.004070,0.016121,0.001448,45,594.02826,0.0,91.79825,1.092114,0
46,918383,548,0.230510,0.0,0.044912,0.513687,0.00327,24,594.02826,0.0,91.79825,1.092114,0
47,918383,786,0.261841,0.0,0.064066,0.431985,0.010507,12,594.02826,0.0,91.79825,1.092114,0
48,918383,815,0.083969,0.0,0.010714,0.174255,0.002867,29,594.02826,0.0,91.79825,1.092114,0
49,918383,851,0.138455,0.0,0.021227,0.308970,0.001094,57,594.02826,0.0,91.79825,1.092114,0
50,918383,1017,0.041324,0.0,0.003786,0.011342,0.000758,81,594.02826,0.0,91.79825,1.092114,0
51,918383,1028,0.044541,0.0,0.003460,0.014017,0.000581,100,594.02826,0.0,91.79825,1.092114,0
52,918383,1139,0.044117,0.0,0.003477,0.000000,0.000788,78,594.02826,0.0,91.79825,1.092114,0


In [95]:
dataset

,Session_Id,Item_ID,Score_UCF_W,Score_ICBF_W,Score_Graph_W,Score_ICF_W,score_LSTM,rank_LSTM,Max_Score_UCF_W,Max_Score_ICBF_W,Max_Score_Graph_W,Max_Score_ICF_W,target
43,918383,55,0.054177,0.0,0.005355,0.018442,0.002479,31,594.028260,0.0,91.798250,1.092114,0
44,918383,189,0.129413,0.0,0.020602,0.278846,0.001137,54,594.028260,0.0,91.798250,1.092114,0
45,918383,379,0.046153,0.0,0.004070,0.016121,0.001448,45,594.028260,0.0,91.798250,1.092114,0
46,918383,548,0.230510,0.0,0.044912,0.513687,0.00327,24,594.028260,0.0,91.798250,1.092114,0
47,918383,786,0.261841,0.0,0.064066,0.431985,0.010507,12,594.028260,0.0,91.798250,1.092114,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4617442,999998,4175,0.000000,0.0,0.000000,0.024474,0.001589,68,81.306076,0.0,2.537715,1.362994,0
4617443,999998,4435,0.000000,0.0,0.000000,0.145432,0.001253,91,81.306076,0.0,2.537715,1.362994,0
4617444,999998,4456,0.000000,0.0,0.000000,0.138890,0.001881,55,81.306076,0.0,2.537715,1.362994,0
4617445,999998,4463,0.000000,0.0,0.000000,0.144827,0.001186,97,81.306076,0.0,2.537715,1.362994,0


In [96]:
dataset_macro_features = pd.read_csv('drive/MyDrive/recsys2022-main/dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [99]:
dataset.score_LSTM = dataset.score_LSTM.astype('float')

In [100]:
from sklearn.model_selection import KFold
import lightgbm
import os
k=10


score=0
kf = KFold(n_splits=k)
index=0
for train_index, valid_index in kf.split(train):

    train_sessions=train["Session_Id"][train_index].tolist()
    valid_sessions=train["Session_Id"][valid_index].tolist()

    train_set=dataset[dataset["Session_Id"].isin(train_sessions)]
    valid_set=dataset[dataset["Session_Id"].isin(valid_sessions)]

    train_set=train_set.sort_values("Session_Id")
    valid_set=valid_set.sort_values("Session_Id")

    qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()

    X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_train=train_set["target"]

    X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_valid=valid_set["target"]

    model = lightgbm.LGBMRanker(
    objective="lambdarank",
    #objective="rank_xendcg",
    #metric="mrr",
    n_estimators=2000,
    lambdarank_truncation_level=103,
    reg_alpha=1e-4,
    learning_rate=0.05,
    min_child_samples=400,
    subsample=0.8,
    subsample_freq=20
    )
    ranker=model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        eval_at=(100),
        callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
        #verbose=10,
        eval_metric=["map","ndcg"]
    )
    ranker.booster_.save_model(f'drive/MyDrive/recsys2022-main/rankers/model_{index}.txt')
    #Get maximum val score
    score+=np.max(ranker._evals_result["valid_0"]["map@100"])
    index+=1

[LightGBM] [Info] Total groups: 33827, total data: 2084874
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.213395 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3805
[LightGBM] [Info] Number of data points in the train set: 2084874, number of used features: 17
[LightGBM] [Info] Total groups: 4111, total data: 262486
[1]	valid_0's map@100: 0.327588	valid_0's ndcg@100: 0.465609
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.334544	valid_0's ndcg@100: 0.471308
[3]	valid_0's map@100: 0.338827	valid_0's ndcg@100: 0.474694
[4]	valid_0's map@100: 0.338501	valid_0's ndcg@100: 0.474522
[5]	valid_0's map@100: 0.337619	valid_0's ndcg@100: 0.473962
[6]	valid_0's map@100: 0.339557	valid_0's ndcg@100: 0.475411
[7]	valid_0's map@100: 0.33926	valid_0's ndcg@100: 0.475178
[8]	valid_0's map@100: 0.338695	valid_0's 

In [101]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.38653423094804457


In [102]:
print("normalized score ", final_performcance*HITRATE)

normalized score  0.1797099957562122
